In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.preprocessing import MinMaxScaler,LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn import metrics#参数，roc和auc
from sklearn.preprocessing import label_binarize#二值化
import joblib
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
names0 = ["AmismatchRate","A_Qrate","UmismatchRate","U_Qrate","GmismatchRate","G_Qrate","CmismatchRate","C_Qrate","del_rate","insert_rate","label0"]#df.columns
names = ["AmismatchRate","A_Qrate","UmismatchRate","U_Qrate","GmismatchRate","G_Qrate","CmismatchRate","C_Qrate","del_rate","insert_rate","label"]#df.columns
path1="./new.paper-unlabled.csv"
path2="./new.treat.csv"
# path1="~/project/t3_ontnewrna/output/paper-unlabled/TrainData/CrandomForestTrainData.csv"
# path2="~/project/t3_ontnewrna/output/OV-FTO-2-0603/TrainData/TrandomForestTrainData.csv"
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df1["label0"]=0
df2["label0"]=1
df=pd.concat([df1,df2])
df = df[df["coverage"]>=0.8]
df = df[names0]
df=df.replace([np.inf, -np.inf], np.nan)
df.eval("""
Am_AQ = AmismatchRate*A_Qrate
Am_Um = AmismatchRate*UmismatchRate
Am_UQ = AmismatchRate*U_Qrate
Am_Gm = AmismatchRate*GmismatchRate
Am_GQ = AmismatchRate*G_Qrate
Am_Cm = AmismatchRate*CmismatchRate
Am_CQ = AmismatchRate*C_Qrate
Am_del = AmismatchRate*del_rate
Am_ins = AmismatchRate*insert_rate
AQ_Um = A_Qrate*UmismatchRate
AQ_UQ = A_Qrate*U_Qrate
AQ_Gm = A_Qrate*GmismatchRate
AQ_GQ = A_Qrate*G_Qrate
AQ_Cm = A_Qrate*CmismatchRate
AQ_CQ = A_Qrate*C_Qrate
AQ_del = A_Qrate*del_rate
AQ_ins = A_Qrate*insert_rate
Um_UQ = UmismatchRate*U_Qrate
Um_Gm = UmismatchRate*GmismatchRate
Um_GQ = UmismatchRate*G_Qrate
Um_Cm = UmismatchRate*CmismatchRate
Um_CQ = UmismatchRate*C_Qrate
Um_del = UmismatchRate*del_rate
Um_ins = UmismatchRate*insert_rate
UQ_Gm = U_Qrate*GmismatchRate
UQ_GQ = U_Qrate*G_Qrate
UQ_Cm = U_Qrate*CmismatchRate
UQ_CQ = U_Qrate*C_Qrate
UQ_del = U_Qrate*del_rate
UQ_ins = U_Qrate*insert_rate
Gm_GQ = GmismatchRate*G_Qrate
Gm_Cm = GmismatchRate*CmismatchRate
Gm_CQ = GmismatchRate*C_Qrate
Gm_del = GmismatchRate*del_rate
Gm_ins = GmismatchRate*insert_rate
GQ_Cm = G_Qrate*CmismatchRate
GQ_CQ = G_Qrate*C_Qrate
GQ_del = G_Qrate*del_rate
GQ_ins = G_Qrate*insert_rate
Cm_CQ = CmismatchRate*C_Qrate
Cm_del = CmismatchRate*del_rate
Cm_ins = CmismatchRate*insert_rate
CQ_del = C_Qrate*del_rate
CQ_ins = C_Qrate*insert_rate
del_ins = del_rate*insert_rate
label = label0
""",inplace=True)
# df.eval("label = label1",inplace=True)
df=df.drop("label0",axis=1)
print(df.shape)
df

(1628237, 56)


,AmismatchRate,A_Qrate,UmismatchRate,U_Qrate,GmismatchRate,G_Qrate,CmismatchRate,C_Qrate,del_rate,insert_rate,...,GQ_CQ,GQ_del,GQ_ins,Cm_CQ,Cm_del,Cm_ins,CQ_del,CQ_ins,del_ins,label
7,0.0125,0.0550,0.0063,0.0238,0.0047,0.0099,0.0141,0.0468,0.1033,0.0360,...,0.000463,0.001023,0.000356,0.000660,0.001457,0.000508,0.004834,0.001685,0.003719,0
8,0.0033,0.0181,0.0000,0.0000,0.0166,0.0397,0.0066,0.0220,0.0399,0.0498,...,0.000873,0.001584,0.001977,0.000145,0.000263,0.000329,0.000878,0.001096,0.001987,0
9,0.0015,0.0041,0.0167,0.0761,0.0091,0.0231,0.0091,0.0303,0.0621,0.0258,...,0.000700,0.001435,0.000596,0.000276,0.000565,0.000235,0.001882,0.000782,0.001602,0
10,0.0062,0.0212,0.0062,0.0257,0.0109,0.0284,0.0140,0.0454,0.0884,0.0295,...,0.001289,0.002511,0.000838,0.000636,0.001238,0.000413,0.004013,0.001339,0.002608,0
12,0.0163,0.0746,0.0245,0.1150,0.0245,0.0651,0.0310,0.1109,0.1240,0.0424,...,0.007220,0.008072,0.002760,0.003438,0.003844,0.001314,0.013752,0.004702,0.005258,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888632,0.0347,0.0949,0.0146,0.0586,0.0037,0.0224,0.0274,0.0772,0.1645,0.0786,...,0.001729,0.003685,0.001761,0.002115,0.004507,0.002154,0.012699,0.006068,0.012930,1
888639,0.0202,0.1093,0.0289,0.1585,0.0190,0.0420,0.0166,0.0411,0.2755,0.1042,...,0.001726,0.011571,0.004376,0.000682,0.004573,0.001730,0.011323,0.004283,0.028707,1
888640,0.0136,0.0592,0.0195,0.0946,0.0161,0.0362,0.0187,0.0404,0.1511,0.0475,...,0.001462,0.005470,0.001720,0.000755,0.002826,0.000888,0.006104,0.001919,0.007177,1
888653,0.0246,0.0965,0.0320,0.1712,0.0269,0.0771,0.0326,0.0872,0.1046,0.0394,...,0.006723,0.008065,0.003038,0.002843,0.003410,0.001284,0.009121,0.003436,0.004121,1


In [4]:
#.划分测试集合训练集

x=df.iloc[:,0:-1]
y=df.iloc[:,-1:]

x=x.replace("inf", np.NAN)

#通过平局值来替换nan
#imputer = SimpleImputer(missing_values="NaN")
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
x=imputer.fit_transform(x,y)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
print("训练样本数量:%d,特征属性数目:%d,目标属性数目:%d"%(x_train.shape[0],x_train.shape[1],y_train.shape[1]))

训练样本数量:1302589,特征属性数目:55,目标属性数目:1


In [5]:
# ss = MinMaxScaler()
#x_train
# y_train
ss = MinMaxScaler()
x_train2 = x_train
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)
x_train2 = ss.transform(x_train2)
y12 = []
for i in y_test["label"]:
    y12.append(i)

In [6]:
rf = RandomForestClassifier(n_estimators=200,criterion="gini",max_depth=30,oob_score=True,random_state=1)
rf.fit(x_train,y_train)
#joblib.dump(rf, "test.pkl")
score = rf.score(x_test,y_test)
score2 = rf.score(x_train2,y_train)
#score2 = rf.socre(x_train,y_train)
print("准确率：%.2f%%"%(score*100))
print("准确率：%.2f%%"%(score2*100))
cv_results = cross_validate(rf,x_train,y_train,return_train_score=True)
print(cv_results["train_score"].mean())
print(cv_results["test_score"].mean())
forest_y_score1 = rf.predict(x_test)
forest_y_score1

C:\Users\panda\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


准确率：88.28%
准确率：99.64%


KeyboardInterrupt: 

In [7]:
feat_labels = df.columns[0:-1]
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(x_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 56, feat_labels[indices[f]], importances[indices[f]]))

 1) UmismatchRate                                            0.096517
 2) Um_UQ                                                    0.073262
 3) Am_Um                                                    0.069836
 4) Um_del                                                   0.046196
 5) Um_ins                                                   0.037745
 6) U_Qrate                                                  0.037229
 7) Um_Gm                                                    0.033641
 8) Am_UQ                                                    0.029904
 9) AQ_Um                                                    0.029060
10) UQ_del                                                   0.022932
11) AQ_CQ                                                    0.021852
12) UQ_GQ                                                    0.021512
13) Um_Cm                                                    0.021315
14) AQ_UQ                                                    0.019242
15) UQ_ins          

In [7]:
forest_y_score = rf.predict_proba(x_test)
p1,p2=[],[]
for j in forest_y_score:
    p1.append(j[0])
    p2.append(j[1])
with open("x_test.csv","w") as opf:
    opf.write("AmismatchRate,UmismatchRate,GmismatchRate,CmismatchRate,label,predict,prob1,prob2\n")
    for i in x_test:
        opf.write(",".join(map(str, i))+"\n")
# xtest = pd.read_csv("./x_test.csv")
# xtest["label"] = y12
# xtest["predict"] = forest_y_score1
# xtest["prob1"] = p1
# xtest["prob2"] = p2
# xtest.to_csv("xtest.csv",index=False,sep=',')
# print(x_test)
# print(x_test.shape)
# print(forest_y_score)
# print(forest_y_score.shape)
#print(y_test[names[-1]])
# forest_y_score = rf.predict_proba(x_test)
# print(forest_y_score)
#计算roc和auc
forest_fpr1, forest_tpr1, _ = metrics.roc_curve(label_binarize(y_test[names[-1]],classes=[0,1,2]).T[0:-1].T.ravel(), forest_y_score.ravel())
#AUC值
auc1 = metrics.auc(forest_fpr1, forest_tpr1)
print ("5EU目标属性AUC值：", auc1)

KeyError: 'label1'

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(8, 6), facecolor='w')
plt.plot(forest_fpr1,forest_tpr1,c='r',lw=2,label=u'5EU目标属性,AUC=%.3f' % auc1)
plt.plot((0,1),(0,1),c='#a0a0a0',lw=2,ls='--')
plt.xlim(-0.001, 1.001)
plt.ylim(-0.001, 1.001)
plt.xticks(np.arange(0, 1.1, 0.1))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('False Positive Rate(FPR)', fontsize=16)
plt.ylabel('True Positive Rate(TPR)', fontsize=16)
plt.grid(b=True, ls=':')
plt.legend(loc='lower right', fancybox=True, framealpha=0.8, fontsize=12)
plt.title(u'随机森林多目标属性分类ROC曲线', fontsize=18)
plt.show()